In [28]:
import ximea
print(ximea.__version__)

4.16.00


In [29]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
#from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# change the following to %matplotlib notebook for interactive plotting
%matplotlib qt5


# Optionally, tweak styles.
mpl.rc('figure',  figsize=(10, 6)) # defaut figure size
mpl.rc('image', cmap='gray')

import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience

import pims # to read the image stack
import trackpy as tp # for particle tracking
import time
import skimage as sk
from skimage import filters, measure

from IPython import display
from IPython.display import clear_output
from IPython.display import display
import ipywidgets as widgets # used for the progress bar

import time

from ximea import xiapi
import cv2

In [30]:
features = pd.DataFrame()
plt.figure(1,figsize=(6, 4))
plt.xlabel("Time (s)")
plt.ylabel("Mean Diameter (microns)")
#plt.ylim(0,20)
plt.ion()
plt.show(1)

In [31]:
plt.figure(2,figsize=(5, 5))
ax = plt.axes()
plt.xlabel('X axis')
plt.ylabel('Y axis')
plt.ion()
plt.show(2)

In [32]:
def thresholding(images):
    
    #gray = cv2.cvtColor(images, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(images, (5,5), 0)
    maxValue = 255
    
    adaptiveMethod = cv2.ADAPTIVE_THRESH_GAUSSIAN_C#cv2.ADAPTIVE_THRESH_MEAN_C #cv2.ADAPTIVE_THRESH_GAUSSIAN_C
    thresholdType = cv2.THRESH_BINARY#cv2.THRESH_BINARY #cv2.THRESH_BINARY_INV
    blockSize = 55 #odd number like 3,5,7,9,11
    C = 3 # constant to be subtracted
   
    binary = cv2.adaptiveThreshold(gray, maxValue, adaptiveMethod, thresholdType, blockSize, C) 
    test_detection(binary,gray)
    return;

In [33]:
def test_detection(binary,gray):
    
    global features,ax, rect, t0, diameter 
    
    diameter = np.array([])
    grey = gray
    img_example = ~binary
    
    image_dimensions =len(img_example)
    

    # Label elements on the picture
    label_image = sk.measure.label(img_example)
    
    #fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 8))
    #plt.xlabel('X axis')
    #plt.ylabel('Y axis')
    
    
    
    for region in sk.measure.regionprops(label_image, intensity_image=img_example):
        
        if region.area < 200 or region.area > 800:# Everywhere, skip small and large areas
            continue
        
        if region.eccentricity>0.8:
            continue
        
        # exclude the objects that touches the borders
        minr, minc, maxr, maxc = region.bbox
        if minr==0 or minc==0 or maxr==image_dimensions or maxc==image_dimensions:
            continue
               
        
        elongation = float(region.major_axis_length)/float(region.minor_axis_length)
        if elongation>3:
            continue
        
        # Draw rectangle which survived to the criterions
        minr, minc, maxr, maxc = region.bbox
        rect = mpl.patches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                              fill=False, edgecolor='red', linewidth=1)
        
        ax.add_patch(rect)
        
       
        diameter = np.append(region.equivalent_diameter,diameter)
        
        features = features.append([{'time':int(time.time() - t0),
                                     'area':region.area,
                                     'equiv_diameter':region.equivalent_diameter},])
   
    
    
   
    plt.figure(2)
    
    ax.imshow(grey)
    

        
    plt.draw()
    plt.pause(0.001)

    
    
    
    
    plt.figure(1)
    plt.scatter((int(time.time() - t0)),(np.mean(diameter)*0.55))     
    plt.draw()
    plt.pause(0.001)
    
   
    
    return;

In [34]:
global rect, diameter

#create instance for first connected camera 
cam = xiapi.Camera()


#start communication
print('Opening first camera...')
cam.open_device()

#settings
#set exposure time in micro-seconds
cam.set_exposure(1500)

cam.set_gain(6)

cam.set_width(2048)

cam.set_height(90)

cam.set_offsetX(0)

cam.set_offsetY(958)


#create instance of Image to store image data and metadata
img = xiapi.Image()

#start data acquisition
print('Starting data acquisition...')
cam.start_acquisition()

global t0, features

images = np.array(())

   

try:
    print('Starting video. Press I,I to exit.')
    t0 = time.time()
    j = time.time()
    
    
    while True:
        #get data and pass them from camera to img
        cam.get_image(img)
        
        #create numpy array with data from camera. Dimensions of the array are 
        #determined by imgdataformat
        data = img.get_image_data_numpy()
      
        if (time.time() - j > 5.0):
            
            j = time.time()
            
            images  = (data)
            thresholding(images)
            ax.clear()
            del diameter
            
            
            
            
        #show acquired image with time since the beginning of acquisition
        font = cv2.FONT_HERSHEY_SIMPLEX
        
        text = ['{:5.2f}'.format(time.time()-t0), str(int(cam.get_framerate()))]
        
        cv2.putText(
            data, text[0], (10,60), font, 1, (255, 255, 255), 2
            )
        cv2.putText(
            data, text[1]+'fps', (300,60), font, 1, (255, 255, 255), 2
            )
        
        
        cv2.imshow('XiCAM example', data)
    
        cv2.waitKey(1)
        
except KeyboardInterrupt:
    cv2.destroyAllWindows()


#stop data acquisition
print('Stopping acquisition...')
cam.stop_acquisition()

#stop communication
cam.close_device()

print('Done.')


Opening first camera...
Starting data acquisition...
Starting video. Press I,I to exit.


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Stopping acquisition...
Done.


In [39]:
features.to_csv('C:/XIMEA/New Folder/test2 25-10-2018.csv')

In [38]:
plt.savefig('25-10-2018 test 2.png',dpi=300)